In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Importing Libraries

In [13]:
!pip install gensim

In [15]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

Data Import

In [2]:
df = pd.read_csv("../input/cardataset/data.csv")
df.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


### Data Preprocessing

For using Gensim word2vec we need **list of lists**. So we need to convert each text as list of words, i.e. every document is contained in a list and every list contains lists of tokens of that document.

1. Creating a new column for make and model

In [3]:
df['Maker_Model'] = df['Make'] + " " + df['Model']
df.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP,Maker_Model
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135,BMW 1 Series M
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650,BMW 1 Series
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350,BMW 1 Series
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450,BMW 1 Series
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500,BMW 1 Series


In [4]:
# Selecting different features from original dataset to form a new dataframe
df1 = df[['Engine Fuel Type','Transmission Type','Driven_Wheels','Market Category','Vehicle Size', 'Vehicle Style', 'Maker_Model']]

In [5]:
# For each row, combine all the columns into one column
df2 = df1.apply(lambda x: ', '.join(x.astype(str)),axis=1)

In [7]:
# Making df2 as pandas dataframe
df_clean = pd.DataFrame({'clean':df2})

In [9]:
# Creating the list of list format of the custom corpus for gensim modeling
sent = [row.split(', ') for row in df_clean['clean']]

### Gensim word2vec Model Training

In [17]:
model = Word2Vec(sent, min_count = 1, vector_size = 50, workers = 3,
                window = 3, sg = 1,)

In [19]:
model.wv['Toyota Camry']

array([-0.0518703 , -0.10816386, -0.02107531, -0.08490771,  0.00509961,
       -0.0013148 ,  0.0874439 ,  0.34113494,  0.00268544,  0.03647607,
        0.08414687,  0.0245909 ,  0.26280656,  0.11261292,  0.07668688,
        0.0766089 ,  0.15393433,  0.37065253, -0.12054419, -0.22090882,
        0.0632963 , -0.03235442,  0.3308322 ,  0.02973773,  0.13790028,
       -0.05923363, -0.04799149,  0.22648114, -0.08754437, -0.16727038,
       -0.11358853, -0.0058153 ,  0.02961314,  0.05375863,  0.12379092,
       -0.15700018,  0.28571716, -0.00493271,  0.07494245,  0.00568911,
        0.16743974,  0.04226163,  0.01087106,  0.0690635 ,  0.09469088,
       -0.1024166 , -0.01701424, -0.14819181, -0.03129958, -0.11495144],
      dtype=float32)

### Computing Similarities

In [21]:
model.wv.similarity('Porsche 718 Cayman', 'Nissan Van')

0.80506575

In [22]:
model.wv.similarity('Porsche 718 Cayman', 'Mercedes-Benz SLK-Class')

0.9552743

'n' most similar cars

In [23]:
model.wv.most_similar('Mercedes-Benz SLK-Class')[:5]

[('BMW Z4', 0.9897493124008179),
 ('Mercedes-Benz SL-Class', 0.9897083640098572),
 ('Infiniti G Convertible', 0.9888522028923035),
 ('Nissan GT-R', 0.9879506230354309),
 ('Bentley Continental GT', 0.9877930879592896)]

Finding Cosine Similarities

In [25]:
from sklearn.metrics.pairwise import cosine_distances

In [41]:
def cosine_distance (model, word,target_list, num) :
    cosine_dict ={}
    word_list = []
    a = model.wv[word]
    for item in target_list :
        if item != word :
            b = model.wv[item]
            cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
            cosine_dict[item] = cos_sim
    dist_sort=sorted(cosine_dict.items(), key=lambda dist: dist[1],reverse = True) ## in Descedning order 
    for item in dist_sort:
        word_list.append((item[0], item[1]))
    return word_list[0:num]

In [42]:
# only get the unique Maker_Model
Maker_Model = list(df.Maker_Model.unique()) 

In [43]:
# Show the most similar Mercedes-Benz SLK-Class by cosine distance 
cosine_distance (model,'Mercedes-Benz SLK-Class',Maker_Model,5)

[('BMW Z4', 0.9897494),
 ('Mercedes-Benz SL-Class', 0.9897084),
 ('Infiniti G Convertible', 0.98885226),
 ('Nissan GT-R', 0.98795056),
 ('Bentley Continental GT', 0.98779297)]